# 1. Initialization

In [ ]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [ ]:
import sys
from dotenv import load_dotenv
import os
load_dotenv()


## 1.1 Import Functions

In [ ]:
import uuid
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="Trying to unpickle estimator StandardScaler from version")
from configs.history_data_crawlers_config import root_path
from datetime import timedelta
from realtime.realtime_dataset_functions import dataset_gen_realtime_loop, Error502_handling
import numpy as np
run_id = str(uuid.uuid4())

from realtime.realtime_utils import (
    add_RSI_to_realtime_dataset,
    add_real_time_candles,
    add_candle_fe,
    add_fe_cndl_shift_fe_realtime_run,
    add_fe_win_realtime_run,
    add_fe_time_realtime_run,
    add_fe_market_close_realtime_run,
    get_coinex_time_now,
    sleep_until_next_run,
    predict_realtime,
    add_ohlcv_non_w_to_realtime_dataset,
    add_ohlcv_w_to_realtime_dataset,
    add_78_to_realtime_dataset,
    add_trade_features_to_realtime_dataset
)


In [ ]:
now = get_coinex_time_now()
type(now)

## 1.2 Initialize Reporter (Telegram)

Configure your telegram bot variables. You need to set `TELEGRAM_CHAT_ID`, `TELEGRAM_CHAT_ID` and `TELEGRAM_BOT_TOKEN_SIGNAL` in the .env file. See [this tutorial](https://gist.github.com/nafiesl/4ad622f344cd1dc3bb1ecbe468ff9f8a).

In [ ]:
from utils.telegram_functions import log_agent
from datetime import datetime
import pytz
import os


bot_token = os.getenv('TELEGRAM_BOT_TOKEN')
bot_token_signal = os.getenv('TELEGRAM_BOT_TOKEN_SIGNAL')
chat_id = os.getenv('TELEGRAM_CHAT_ID')
bot_token_signal_zaker = os.getenv('TELEGRAM_BOT_TOKEN_SIGNAL_ZAKER')
chat_id_zaker = os.getenv('TELEGRAM_CHAT_ID_ZAKER')
now = datetime.now(pytz.timezone('Asia/Tehran'))
date_string = now.strftime("%d/%m/%Y %H:%M:%S")
bot = log_agent(bot_token=bot_token,chat_id=chat_id, PRINT=True, TELEGRAM=True)
bot_signal = log_agent(bot_token=bot_token_signal,chat_id=chat_id, PRINT=True, TELEGRAM=True)
bot.send_message(text="🚩"*15)
bot.send_message(text="👇"*15)
bot.send_message(text=f"start date_time ={date_string} - tehran time")  
bot_signal.send_message(text=f"start date_time ={date_string} - tehran time")  


## 1.3 Load Model & Set Strategy

In [ ]:
import pickle
from realtime.realtime_utils import load_models

symbol_list = ['AAVEUSDT',]

till_date = '2025-09-30'
max_open_positions = 20

models_list = load_models(symbol_list, till_date, max_open_positions, bot)

In [ ]:
max_open_positions_all = 0
for model in models_list:
    max_open_positions_all += 1
max_open_positions_all *= 20

## 1.4 Set Feature Creation Config

### General Config

In [ ]:
from configs.feature_configs_general import generate_general_config
feature_config = generate_general_config()

In [ ]:
feature_config.keys()

## 1.5 Initialize Binance & Coinex

In [ ]:


from trade_execution.coinex_trade_execution_functions import (
    CoinexRequestsClient,
    cal_candle_time_now,
    get_spot_balance,
    check_active_positions_for_time_force_close,
    close_or_extend_expired_trades,
    open_position,
    check_tp_sl_positions,
    close_and_exit,
    cancel_all_orders,
    get_open_stop_orders,
    get_open_orders,
    send_stop_order,
    cancel_stop_order,
    count_active_positions
)



bot.send_message(text="--> coinex connected.")



## 1.6 Initialize Dataset

In [ ]:
"""
create dataset for the first time.

"""

data_size_in_days_ohlcv = 60
data_size_in_days_trade = 3.5
data_size_in_minutes_trade = data_size_in_days_trade*1440


fe_functions = [
    add_real_time_candles,
    add_RSI_to_realtime_dataset,
    add_fe_cndl_shift_fe_realtime_run,
    add_candle_fe,
    add_fe_time_realtime_run,
    add_fe_win_realtime_run,
    add_fe_market_close_realtime_run,
    add_ohlcv_non_w_to_realtime_dataset,
    add_ohlcv_w_to_realtime_dataset,
    add_78_to_realtime_dataset,
    add_trade_features_to_realtime_dataset
]
after_merge_functions = [
  
]


dataset_config = {
    "features": [
        "fe_cndl",
        "fe_RSI",
        "fe_ratio",
        "fe_EMA",
        "fe_SMA",
        "fe_cndl_shift",
        "fe_WIN",
        "fe_ATR",
        "fe_RSTD",
        "fe_time",
        "fe_market_close",
    ]
}

datasetdict_path = f"{root_path}/data/datasetdict.pkl"
if not os.path.exists(datasetdict_path):
    dataset, final_df, crawl_time, shape, len_candle = dataset_gen_realtime_loop(
    'init', fe_functions,feature_config, dataset_config, dataset={}, data_size_in_days_ohlcv=data_size_in_days_ohlcv,\
          data_size_in_days_trade=data_size_in_days_trade,\
          data_size_in_minutes_trade=data_size_in_minutes_trade)

    dataset_dict = {"dataset": dataset,
                    "final_df": final_df,
                    "crawl_time": crawl_time,
                    "shape": shape,
                    "len_candle": len_candle}
    with open(f'{root_path}/data/datasetdict.pkl', "wb") as f:
        pickle.dump(dataset_dict, f)
    del dataset_dict
    bot.send_message(f"--> final_df initialized till {final_df.index[-1].tz_localize('UTC')}.")

else:
    with open(datasetdict_path, "rb") as f:
        dataset_dict = pickle.load(f)
        dataset = dataset_dict["dataset"]
        final_df = dataset_dict["final_df"]
        crawl_time = dataset_dict["crawl_time"]
        shape = dataset_dict["shape"]
        len_candle = dataset_dict["len_candle"]

In [ ]:
while (get_coinex_time_now() - final_df.index[-1].tz_localize('UTC')) >\
      timedelta(minutes=9.5):

    print("indate")
    diff_min = (get_coinex_time_now() - final_df.index[-1].tz_localize('UTC'))
    total_seconds = diff_min.total_seconds()
    hours = int(total_seconds // 3600)
    minutes = int((total_seconds % 3600) // 60) 
    indate_mins = int(np.ceil((hours*60 + minutes)/5) * 5)
    dataset, final_df, crawl_time, shape, len_candle = dataset_gen_realtime_loop(
        'indate',
        fe_functions,
        feature_config,
        dataset_config,
        dataset,
        data_size_in_days_ohlcv=data_size_in_days_ohlcv,
        data_size_in_days_trade=data_size_in_days_trade,
        data_size_in_minutes_trade=data_size_in_minutes_trade,
        shape=shape,
        len_candle=len_candle,
        indate_mins = indate_mins
    )
    dataset_dict = {"dataset": dataset,
                    "final_df": final_df,
                    "crawl_time": crawl_time,
                    "shape": shape,
                    "len_candle": len_candle}
    with open(f'{root_path}/data/datasetdict.pkl', "wb") as f:
        pickle.dump(dataset_dict, f)
    del dataset_dict
    bot.send_message(f"--> final_df initialized till {final_df.index[-1].tz_localize('UTC')}.")

# 2. Realtime Process

In [ ]:
import pandas as pd
import os

prediction_data_path = f'{root_path}/data/realtime_trade_prediction_data.parquet'  # Replace with your actual file path

try:
  # Attempt to read the DataFrame from the file
  if os.path.exists(prediction_data_path):
    print("--> read realtime_trade_prediction_data:")
    preds_df = pd.read_parquet(prediction_data_path)
  else:
    print("--> no realtime_trade_prediction_data file. create an empty one:")
    preds_df = pd.DataFrame()
except Exception as e:
  print(f"An error occurred while reading the file: {e}")
  preds_df = pd.DataFrame()

predictions = {}

In [ ]:
"""
main loop predict

"""
import time as tt
import traceback
from datetime import datetime
import pickle
import os
import gc
import subprocess


run_every = 5 # run every X rounded minute
offset_seconds = 1

bot.send_message(f"--> start the realtime loop, run every {run_every} minuts.")
while True:
    try:
        with open(f"{root_path}/data/accounts.pkl", "rb") as f:
            accounts = pickle.load(f)
        # Load all_trades if the file exists
        all_trades_path = f"{root_path}/data/all_trades.pkl"
        if os.path.exists(all_trades_path):
            with open(all_trades_path, "rb") as f:
                all_trades = pickle.load(f)
        else:
            all_trades = {} 
            for account in accounts.keys():
                all_trades[account] = {}
        
        for account in accounts.keys():
            if len(all_trades[account]) > 1000:
                all_trades[account] = dict(list(all_trades[account].items())[-1000:])

        
        sleep_until_next_run(run_every = run_every , offset_seconds = offset_seconds,reporter=bot)
        # bot.send_message(f"============")
        bot.send_message(f"📈 "*5)

        #?? get data
        dataset, final_df, crawl_time, shape, len_candle = dataset_gen_realtime_loop(
        'update',
        fe_functions,
        feature_config,
        dataset_config,
        dataset,
        data_size_in_days_ohlcv=data_size_in_days_ohlcv,
        data_size_in_days_trade=data_size_in_days_trade,
        data_size_in_minutes_trade=data_size_in_minutes_trade,
        shape=shape,
        len_candle=len_candle,
        )
        dataset_dict = {"dataset": dataset,
                    "final_df": final_df,
                    "crawl_time": crawl_time,
                    "shape": shape,
                    "len_candle": len_candle}
        with open(f'{root_path}/data/datasetdict.pkl', "wb") as f:
            pickle.dump(dataset_dict, f)
        bot.send_message(f"--> data updated.")
        del dataset_dict

        expired_trades = {}
        
        #?? check for timing
        for account in accounts.keys():
            access_id = accounts[account]["access_id"]
            secret_key = accounts[account]["secret_key"]
            coinex_request = CoinexRequestsClient(access_id, secret_key)
            expired_trades_list = []
            
            for coin in symbol_list:
                # for model in coin:
                expired_trades_sublist = []
                symbol = coin
                all_trades[account] = check_tp_sl_positions(coinex_request, all_trades[account], symbol)
                # bot.send_message(f"--> func check_tp_sl_positions {symbol} done.")
                all_trades[account], expired_trades_sublist = check_active_positions_for_time_force_close(account, coinex_request, all_trades[account], symbol, bot_signal)
                # bot.send_message(f"--> func check_active_positions_for_time_force_close {symbol} done.")
                expired_trades_list += expired_trades_sublist
            
            expired_trades[account] = expired_trades_list
        #?? predict
        predictions, preds_df = predict_realtime(models_list,predictions,preds_df,crawl_time,final_df,reporter=bot)

        #?? trade
        candle_now = cal_candle_time_now()
        signals_df = preds_df.loc[(preds_df["_time"] == candle_now ) & (preds_df["model_prediction"] == 1)].drop_duplicates("model_id_name",keep="last")
        
        for account in accounts.keys():
            access_id = accounts[account]["access_id"]
            secret_key = accounts[account]["secret_key"]
            coinex_request = CoinexRequestsClient(access_id, secret_key)
            all_trades[account], signal_dff = close_or_extend_expired_trades(account, coinex_request, signals_df, expired_trades[account], all_trades[account], bot_signal)
            # bot.send_message(f"--> func close_or_extend_expired_trades {account} done.")
            if signal_dff.shape[0]>0:
                # bot.send_message(f"--> shape signals_df: {signals_df.shape[0]}.")
                pred_time = signal_dff.iloc[0]['predict_time'].strftime("%Y-%m-%d %H:%M:%S")
                # bot.send_message(f"--> predict_time: {pred_time}.")
                bot_signal.send_message(f"--> predict_time: {pred_time}.")
                # bot_signal_zaker.send_message(f"--> predict_time: {pred_time}.")
            for _,row in signal_dff.iterrows():
                trade_side = row["strategy_trade_mode"]
                symbol = row["strategy_target_symbol"].upper()
                tp_percent = row["strategy_take_profit"]
                sl_percent = row["strategy_stop_loss"]
                look_ahead = row["strategy_look_ahead"] ##?? in minutes
                max_open_positions = row["strategy_max_open_positions"]
                last_candle_close_price = dataset["st_one"][symbol].loc[dataset["st_one"][symbol]["_time"]==cal_candle_time_now()]["close"].values[0]
                with open(f'{root_path}/data/trade_permission.pkl', 'rb') as f:
                    trade_permission = pickle.load(f)
                if trade_permission["P"] == 1:
                    all_trades[account] = open_position(
                        account,
                        coinex_request,
                        bot_signal,
                        trade_side,
                        symbol,
                        tp_percent,
                        sl_percent,
                        last_candle_close_price,
                        base_price_mode="tick_price",
                        all_trades=all_trades[account],
                        max_open_positions=max_open_positions,
                        max_open_positions_all=max_open_positions_all,
                        look_ahead_minutes=look_ahead)
            
            now = get_coinex_time_now()
            if (now.hour == 21 and now.minute > 35 and now.minute < 40) or \
                (now.hour == 9 and now.minute > 35 and now.minute < 40):
                active = {}  # Active positions
                not_active = {}  # Inactive positions

                for order_id, order_info in all_trades[account].items():
                    if order_info.get('position_status_active'):
                        active[order_id] = order_info
                    else:
                        not_active[order_id] = order_info

                # now = get_coinex_time_now()
                now = now.strftime("%Y-%m-%d %H:%M:%S")
                now = now.replace(":", "_")
                # Make sure the directory exists
                directory = f"{root_path}/data/saved_all_trades/{account}"
                os.makedirs(directory, exist_ok=True)

                # Save the pickle file
                with open(f"{directory}/all_trades_{account}_till_{now}.pkl", "wb") as f:
                    pickle.dump(not_active, f)
                all_trades[account] = active
                del active
                # saved_all_trade[account] == True
                bot.send_message(f"--> all_trades saved and flushed for {account}.")

            
        bot.send_message(f"--> loop end.")

        preds_df.to_parquet(f"{root_path}/data/preds_df.parquet")
        with open(all_trades_path, "wb") as f:
            pickle.dump(all_trades, f)
        del all_trades
        gc.collect()
        

    except Exception as e:
        err_str = str(e)
        if "502 Bad Gateway" in err_str or "The request could not be satisfied" in err_str:
        # if True:
            bot.send_message("!!! 502 Bad Gateway detected. Modifying dataset and restarting...")

            Error502_handling()
            tt.sleep(20)
            
            sys.exit(1)  # exit current process so the new one takes over
        
        elif 'invalid slice' in err_str:
            bot.send_message("!!! invalid slice argument detected. Modifying dataset and restarting...")
            
            Error502_handling()
            tt.sleep(20)
            
            sys.exit(1)  # exit current process so the new one takes over

        elif 'Wrong timing' in err_str:
            bot.send_message("!!! Wrong timing. Modifying dataset and restarting...")
            
            Error502_handling()
            tt.sleep(20)
            
            sys.exit(1)  # exit current process so the new one takes over

        else:
            bot.send_message(f"!!! ERROR: {err_str}")
            traceback.print_exc()
            bot.send_message(traceback.format_exc())
            tt.sleep(60)

        